In [1]:
#Quora Dataset
import pandas as pd
df = pd.read_csv("/Users/kushansingh/Desktop/quora_kaggle/train.csv",na_values=[""])

In [2]:
#stemming using PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

df['question_text'] = [x.lower() for x in df['question_text']]
df['question_text'] = [x.translate(str.maketrans('','',string.punctuation)) for x in df['question_text']]
df['question_text'] = [x.translate(str.maketrans('','','1234567890')) for x in df['question_text']]

df['tokenized_sents'] = df.apply(lambda row: word_tokenize(row['question_text']), axis=1)
df.head()

,qid,question_text,target,tokenized_sents
0,00002165364db923c7e6,how did quebec nationalists see their province...,0,"[how, did, quebec, nationalists, see, their, p..."
1,000032939017120e6e44,do you have an adopted dog how would you encou...,0,"[do, you, have, an, adopted, dog, how, would, ..."
2,0000412ca6e4628ce2cf,why does velocity affect time does velocity af...,0,"[why, does, velocity, affect, time, does, velo..."
3,000042bf85aa498cd78e,how did otto von guericke used the magdeburg h...,0,"[how, did, otto, von, guericke, used, the, mag..."
4,0000455dfa3e01eae3af,can i convert montra helicon d to a mountain b...,0,"[can, i, convert, montra, helicon, d, to, a, m..."


In [3]:
# Removing Stop Words
stop_words = set(stopwords.words('english'))
def removeStopWords(word_tokens):
    filtered_tokens = [w for w in word_tokens if w not in stop_words]
    return filtered_tokens
df['tokenized_sents'] = df.apply(lambda row: removeStopWords(row['tokenized_sents']),axis=1)
df.head()

,qid,question_text,target,tokenized_sents
0,00002165364db923c7e6,how did quebec nationalists see their province...,0,"[quebec, nationalists, see, province, nation]"
1,000032939017120e6e44,do you have an adopted dog how would you encou...,0,"[adopted, dog, would, encourage, people, adopt..."
2,0000412ca6e4628ce2cf,why does velocity affect time does velocity af...,0,"[velocity, affect, time, velocity, affect, spa..."
3,000042bf85aa498cd78e,how did otto von guericke used the magdeburg h...,0,"[otto, von, guericke, used, magdeburg, hemisph..."
4,0000455dfa3e01eae3af,can i convert montra helicon d to a mountain b...,0,"[convert, montra, helicon, mountain, bike, cha..."


In [26]:
import os
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = "/Users/kushansingh/glove.6B.100d.txt"
word2vec_output_file = 'glove.txt.word2vec'
glove2word2vec(glove_input_file,word2vec_output_file)


(400000, 100)

In [33]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(word2vec_output_file,binary=False)

In [85]:
df.drop('transformed_tokens',inplace=True,axis=1)


In [86]:
df.head()

,qid,question_text,target,tokenized_sents
0,00002165364db923c7e6,how did quebec nationalists see their province...,0,"[quebec, nationalists, see, province, nation]"
1,000032939017120e6e44,do you have an adopted dog how would you encou...,0,"[adopted, dog, would, encourage, people, adopt..."
2,0000412ca6e4628ce2cf,why does velocity affect time does velocity af...,0,"[velocity, affect, time, velocity, affect, spa..."
3,000042bf85aa498cd78e,how did otto von guericke used the magdeburg h...,0,"[otto, von, guericke, used, magdeburg, hemisph..."
4,0000455dfa3e01eae3af,can i convert montra helicon d to a mountain b...,0,"[convert, montra, helicon, mountain, bike, cha..."


In [87]:
def validTokens(tokens):
    transformed_tokens = [x for x in tokens if x in model.vocab]
    return transformed_tokens
    
df['transformed_tokens'] = df.apply(lambda row: validTokens(row['tokenized_sents']),axis=1)


In [88]:
df.head()

,qid,question_text,target,tokenized_sents,transformed_tokens
0,00002165364db923c7e6,how did quebec nationalists see their province...,0,"[quebec, nationalists, see, province, nation]","[quebec, nationalists, see, province, nation]"
1,000032939017120e6e44,do you have an adopted dog how would you encou...,0,"[adopted, dog, would, encourage, people, adopt...","[adopted, dog, would, encourage, people, adopt..."
2,0000412ca6e4628ce2cf,why does velocity affect time does velocity af...,0,"[velocity, affect, time, velocity, affect, spa...","[velocity, affect, time, velocity, affect, spa..."
3,000042bf85aa498cd78e,how did otto von guericke used the magdeburg h...,0,"[otto, von, guericke, used, magdeburg, hemisph...","[otto, von, guericke, used, magdeburg, hemisph..."
4,0000455dfa3e01eae3af,can i convert montra helicon d to a mountain b...,0,"[convert, montra, helicon, mountain, bike, cha...","[convert, helicon, mountain, bike, changing, t..."


In [94]:
len(df['transformed_tokens'])

1306122

In [102]:
df = df[df['transformed_tokens'].map(lambda d: len(d)) > 0]

In [106]:
df.tail()

,qid,question_text,target,tokenized_sents,transformed_tokens
1306117,ffffcc4e2331aaf1e41e,what other technical skills do you need as a c...,0,"[technical, skills, need, computer, science, u...","[technical, skills, need, computer, science, u..."
1306118,ffffd431801e5a2f4861,does ms in ece have good job prospects in usa ...,0,"[ms, ece, good, job, prospects, usa, like, ind...","[ms, ece, good, job, prospects, usa, like, ind..."
1306119,ffffd48fb36b63db010c,is foam insulation toxic,0,"[foam, insulation, toxic]","[foam, insulation, toxic]"
1306120,ffffec519fa37cf60c78,how can one start a research project based on ...,0,"[one, start, research, project, based, biochem...","[one, start, research, project, based, biochem..."
1306121,ffffed09fedb5088744a,who wins in a battle between a wolverine and a...,0,"[wins, battle, wolverine, puma]","[wins, battle, wolverine, puma]"


In [105]:
df['word_vectors'] = df.apply(lambda row: model[row['transformed_tokens']],axis=1)

Exception: Data must be 1-dimensional